In [29]:
dataset_mapping = {
    'Durum Wheat': ('Identification (I)', 'F1', 'Seed Morphology'),
    'Soybean Seeds': ('Identification (I)', 'F1', 'Seed Morphology'),
    'Mango Leaf Disease': ('Identification (I)', 'F1', 'Foliar Stress'),
    'Bean Leaf Lesions': ('Identification (I)', 'F1', 'Foliar Stress'),
    'Soybean Diseases': ('Identification (I)', 'F1', 'Foliar Stress'),
    'Dangerous Insects': ('Identification (I)', 'F1', 'Invasive Species'),
    'DeepWeeds': ('Identification (I)', 'F1', 'Invasive Species'),
    'Yellow Rust 19': ('Classification (C)', 'NMAE', 'Disease Severity'),
    'IDC': ('Classification (C)', 'NMAE', 'Stress Tolerance'),
    'FUSARIUM 22': ('Classification (C)', 'NMAE', 'Stress Tolerance'),
    'InsectCount': ('Quantification (Q)', 'NMAE', 'Pest'),
    'PlantDoc': ('Quantification (Q)', 'NMAE', 'Disease'),
}


#dataset name, question
question_mapping = {
    'Durum Wheat':'What wheat variety is this?',
    'Soybean Seeds':'What soybean lifecycle stage is this?' ,
    'Mango Leaf Disease': 'What mango leaf disease is present?' ,
    'Bean Leaf Lesions': 'What type of bean leaf lesion is this?',
    'Soybean Diseases': 'What is the type of stress in this soybean?',
    'Dangerous Insects': 'What is the name of this harmful insect?',
    'DeepWeeds':'What is the name of this weed?' ,
    'Yellow Rust 19':'What is the severity of yellow rust disease?' ,
    'IDC':'What is the rating (1-5) of soybean stress severity?' ,
    'FUSARIUM 22':'What is the severity of chickpea fusarium wilt?' ,
    'InsectCount': 'What is the insect count?',
    'PlantDoc': 'What is the diseased leaf percentage?',
}


In [21]:
import os
import pandas as pd
import random
import numpy as np

# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

def get_sample_predictions(n_samples=5):
    results = {}
    datasets = list(dataset_mapping.keys())
    llms = [d for d in os.listdir('results') if os.path.isdir(os.path.join('results', d))]

    for dataset in datasets:
        # Load data for all LLMs
        llm_dfs = {llm: pd.read_csv(f'results/{llm}/{dataset}.csv') for llm in llms}
        
        # Find common filepaths across all LLMs
        common_filepaths = set(llm_dfs[llms[0]]['0'])
        for llm_df in llm_dfs.values():
            common_filepaths &= set(llm_df['0'])
        
        # Filter dataframes to include only common filepaths
        for llm in llms:
            llm_dfs[llm] = llm_dfs[llm][llm_dfs[llm]['0'].isin(common_filepaths)]
        
        # Sample from common filepaths
        sampled_filepaths = random.sample(list(common_filepaths), min(n_samples, len(common_filepaths)))
        
        # Create DataFrame for this dataset
        df_data = []
        for filepath in sampled_filepaths:
            row_data = {'Filepath': filepath}
            # Add ground truth from the first LLM (it's the same for all)
            row_data['Ground_Truth'] = llm_dfs[llms[0]][llm_dfs[llms[0]]['0'] == filepath]['1'].iloc[0]
            for llm, llm_df in llm_dfs.items():
                row = llm_df[llm_df['0'] == filepath].iloc[0]
                row_data.update({
                    f'{llm}_0-shot': row['# of Shots 0'],
                    f'{llm}_8-shot': row['# of Shots 8']
                })
            df_data.append(row_data)
        
        results[dataset] = pd.DataFrame(df_data)

    return results

# Get sample predictions
sample_predictions = get_sample_predictions()

# Print the results
for dataset, df in sample_predictions.items():
    print(f"\n\n=== Dataset: {dataset} ===")
    print(df.to_string(index=False))
    print("-" * 80)



=== Dataset: Durum Wheat ===
                                                                                                                                           Filepath    Ground_Truth Gemini-pro-1.5_0-shot Gemini-pro-1.5_8-shot    GPT-4o_0-shot    GPT-4o_8-shot LLaVA v1.6 34B_0-shot LLaVA v1.6 34B_8-shot Claude-3.5-sonnet_0-shot Claude-3.5-sonnet_8-shot Gemini-flash-1.5_0-shot Gemini-flash-1.5_8-shot Claude-3-haiku_0-shot Claude-3-haiku_8-shot
/Users/muhammadarbabarshad/Downloads/AgEval-datasets/Durum_Wheat_Dataset/Dataset2-Durum Wheat Video Images/processed/Foreign Matters/Frame 0031.jpg Foreign Matters       Foreign Matters       Foreign Matters  Foreign Matters  Foreign Matters       Foreign Matters                   NaN          Foreign Matters          Foreign Matters         Foreign Matters         Foreign Matters      Vitreous Kernels       Foreign Matters
/Users/muhammadarbabarshad/Downloads/AgEval-datasets/Durum_Wheat_Dataset/Dataset2-Durum Wheat Video Images/proces

In [34]:
import os
import random
import pandas as pd

def generate_latex_output(sample_predictions, question_mapping, dataset_mapping, seed=42):
    random.seed(seed)
    
    latex_outputs = []
    files_to_upload = []
    
    for dataset, df in sample_predictions.items():
        # Get dataset information from the mapping
        category, metric, subcategory = dataset_mapping[dataset]
        
        # Get the question for this dataset
        question = question_mapping[dataset]
        
        # Group by ground truth and select one sample from each group
        grouped = df.groupby('Ground_Truth')
        unique_ground_truths = list(grouped.groups.keys())
        
        if len(unique_ground_truths) < 2:
            print(f"Warning: Dataset {dataset} has less than 2 unique ground truth categories.")
            continue
        
        selected_ground_truths = random.sample(unique_ground_truths, 2)
        
        for ground_truth in selected_ground_truths:
            group_df = grouped.get_group(ground_truth)
            sample = group_df.iloc[random.randint(0, len(group_df) - 1)]
            
            # Extract filename from the current path and create new path
            current_path = sample['Filepath']
            filename = os.path.basename(current_path)
            new_path = f"anecdotal/{dataset}/{filename}"
            
            # Add to files to upload
            files_to_upload.append((current_path, new_path))
            
            # Generate the LaTeX output for this sample
            latex_output = f"""
            \\begin{{figure}}[h]
                \\centering
                \\captionsetup{{justification=centering}}
                \\textbf{{\\centering {question}}}
                
                \\vspace{{0.5cm}}
                \\includegraphics[width=0.2\\textwidth]{{{new_path}}}
                \\caption*{{
                    \\begin{{tabular}}{{|p{{0.3\\linewidth}}|p{{0.3\\linewidth}}|p{{0.3\\linewidth}}|}}
                        \\hline
                        \\textbf{{Category}} & \\textbf{{Subcategory}} & \\textbf{{Task}} \\\\
                        \\hline
                        {category} & {subcategory} & {dataset} \\\\
                        \\hline
                    \\end{{tabular}}
                    
                    \\vspace{{0.3cm}}
                    
                    \\textbf{{Ground Truth:}} {sample['Ground_Truth']} \\\\
                    \\vspace{{0.2cm}}
                    \\textbf{{Predictions:}} \\\\
                    \\begin{{tabular}}{{l|c|c}}
                        \\textbf{{Model Name}} & \\textbf{{0 shot}} & \\textbf{{8 shot}} \\\\
                        \\hline
                        Gemini-pro-1.5 & {sample['Gemini-pro-1.5_0-shot']} & {sample['Gemini-pro-1.5_8-shot']} \\\\
                        GPT-4o & {sample['GPT-4o_0-shot']} & {sample['GPT-4o_8-shot']} \\\\
                        LLaVA v1.6 34B & {sample['LLaVA v1.6 34B_0-shot']} & {sample['LLaVA v1.6 34B_8-shot']} \\\\
                        Claude-3.5-sonnet & {sample['Claude-3.5-sonnet_0-shot']} & {sample['Claude-3.5-sonnet_8-shot']} \\\\
                        Gemini-flash-1.5 & {sample['Gemini-flash-1.5_0-shot']} & {sample['Gemini-flash-1.5_8-shot']} \\\\
                        Claude-3-haiku & {sample['Claude-3-haiku_0-shot']} & {sample['Claude-3-haiku_8-shot']}
                    \\end{{tabular}}
                }}
            \\end{{figure}}
            """
            
            latex_outputs.append(latex_output)
    
    return '\n\n'.join(latex_outputs), files_to_upload

# Assuming question_mapping and dataset_mapping are defined as before

# Generate the LaTeX output and get the list of files to upload
latex_output, files_to_upload = generate_latex_output(sample_predictions, question_mapping, dataset_mapping)

# Print the LaTeX output
print("LaTeX Output:")
print(latex_output)

# Print the list of files to upload
print("\nFiles to Upload:")
for original_path, new_path in files_to_upload:
    print(f"Original: {original_path}")
    print(f"New: {new_path}")
    print()

LaTeX Output:

            \begin{figure}[h]
                \centering
                \captionsetup{justification=centering}
                \textbf{\centering What wheat variety is this?}
                
                \vspace{0.5cm}
                \includegraphics[width=0.2\textwidth]{anecdotal/Durum Wheat/Frame 0026.jpg}
                \caption*{
                    \begin{tabular}{|p{0.3\linewidth}|p{0.3\linewidth}|p{0.3\linewidth}|}
                        \hline
                        \textbf{Category} & \textbf{Subcategory} & \textbf{Task} \\
                        \hline
                        Identification (I) & Seed Morphology & Durum Wheat \\
                        \hline
                    \end{tabular}
                    
                    \vspace{0.3cm}
                    
                    \textbf{Ground Truth:} Foreign Matters \\
                    \vspace{0.2cm}
                    \textbf{Predictions:} \\
                    \begin{tabular}{l|c|c}
 

# fixed all

In [39]:
import os
import random
import pandas as pd

def generate_latex_output(sample_predictions, question_mapping, dataset_mapping, seed=42):
    random.seed(seed)
    
    latex_outputs = []
    files_to_upload = []
    
    for dataset, df in sample_predictions.items():
        # Get dataset information from the mapping
        category, metric, subcategory = dataset_mapping[dataset]
        
        # Get the question for this dataset
        question = question_mapping[dataset]
        
        # Group by ground truth and select one sample from each group
        grouped = df.groupby('Ground_Truth')
        unique_ground_truths = list(grouped.groups.keys())
        
        if len(unique_ground_truths) < 2:
            print(f"Warning: Dataset {dataset} has less than 2 unique ground truth categories.")
            continue
        
        selected_ground_truths = random.sample(unique_ground_truths, 2)
        
        for ground_truth in selected_ground_truths:
            group_df = grouped.get_group(ground_truth)
            sample = group_df.iloc[random.randint(0, len(group_df) - 1)]
            
            # Extract filename from the current path and create new path
            current_path = sample['Filepath']
            filename = os.path.basename(current_path)
            new_path = f"anecdotal/{dataset}/{filename}"
            
            # Add to files to upload
            files_to_upload.append((current_path, new_path))
            
            # Generate the LaTeX output for this sample
            latex_output = f"""
            \\begin{{figure}}[h]
                \\centering
                \\captionsetup{{justification=centering}}
                \\fontsize{{8pt}}{{9.6pt}}\\selectfont  % Set font size to 8pt with 1.2 line spacing
                \\textbf{{\\centering {question}}}
                
                \\vspace{{0.3cm}}
                \\includegraphics[width=0.45\\linewidth]{{{new_path}}}
                \\caption*{{
                    \\begin{{tabular}}{{|p{{0.3\\linewidth}}|p{{0.3\\linewidth}}|p{{0.3\\linewidth}}|}}
                        \\hline
                        \\textbf{{Category}} & \\textbf{{Subcategory}} & \\textbf{{Task}} \\\\
                        \\hline
                        {category} & {subcategory} & {dataset} \\\\
                        \\hline
                    \\end{{tabular}}
                    
                    \\vspace{{0.2cm}}
                    
                    \\textbf{{Ground Truth:}} {sample['Ground_Truth']} \\\\
                    \\vspace{{0.1cm}}
                    \\textbf{{Predictions:}} \\\\
                    \\begin{{tabular}}{{l|c|c}}
                        \\textbf{{Model Name}} & \\textbf{{0 shot}} & \\textbf{{8 shot}} \\\\
                        \\hline
                        Gemini-pro-1.5 & {sample['Gemini-pro-1.5_0-shot']} & {sample['Gemini-pro-1.5_8-shot']} \\\\
                        GPT-4o & {sample['GPT-4o_0-shot']} & {sample['GPT-4o_8-shot']} \\\\
                        LLaVA v1.6 34B & {sample['LLaVA v1.6 34B_0-shot']} & {sample['LLaVA v1.6 34B_8-shot']} \\\\
                        Claude-3.5-sonnet & {sample['Claude-3.5-sonnet_0-shot']} & {sample['Claude-3.5-sonnet_8-shot']} \\\\
                        Gemini-flash-1.5 & {sample['Gemini-flash-1.5_0-shot']} & {sample['Gemini-flash-1.5_8-shot']} \\\\
                        Claude-3-haiku & {sample['Claude-3-haiku_0-shot']} & {sample['Claude-3-haiku_8-shot']}
                    \\end{{tabular}}
                }}
            \\end{{figure}}
            """
            
            latex_outputs.append(latex_output)
    
    return '\n\n'.join(latex_outputs), files_to_upload

# Assuming question_mapping and dataset_mapping are defined as before

# Generate the LaTeX output and get the list of files to upload
latex_output, files_to_upload = generate_latex_output(sample_predictions, question_mapping, dataset_mapping)

# Print the LaTeX output
print("LaTeX Output:")
print(latex_output)

# Print the list of files to upload
print("\nFiles to Upload:")
for original_path, new_path in files_to_upload:
    print(f"Original: {original_path}")
    print(f"New: {new_path}")
    print()

LaTeX Output:

            \begin{figure}[h]
                \centering
                \captionsetup{justification=centering}
                \fontsize{8pt}{9.6pt}\selectfont  % Set font size to 8pt with 1.2 line spacing
                \textbf{\centering What wheat variety is this?}
                
                \vspace{0.3cm}
                \includegraphics[width=0.45\linewidth]{anecdotal/Durum Wheat/Frame 0026.jpg}
                \caption*{
                    \begin{tabular}{|p{0.3\linewidth}|p{0.3\linewidth}|p{0.3\linewidth}|}
                        \hline
                        \textbf{Category} & \textbf{Subcategory} & \textbf{Task} \\
                        \hline
                        Identification (I) & Seed Morphology & Durum Wheat \\
                        \hline
                    \end{tabular}
                    
                    \vspace{0.2cm}
                    
                    \textbf{Ground Truth:} Foreign Matters \\
                    \vspace{

# fixed all final


In [41]:
import os
import random
import pandas as pd

def generate_latex_output(sample_predictions, question_mapping, dataset_mapping, seed=42):
    random.seed(seed)
    
    latex_outputs = []
    files_to_upload = []
    
    for dataset, df in sample_predictions.items():
        # Get dataset information from the mapping
        category, metric, subcategory = dataset_mapping[dataset]
        
        # Get the question for this dataset
        question = question_mapping[dataset]
        
        # Group by ground truth and select one sample from each group
        grouped = df.groupby('Ground_Truth')
        unique_ground_truths = list(grouped.groups.keys())
        
        if len(unique_ground_truths) < 2:
            print(f"Warning: Dataset {dataset} has less than 2 unique ground truth categories.")
            continue
        
        selected_ground_truths = random.sample(unique_ground_truths, 2)
        
        for ground_truth in selected_ground_truths:
            group_df = grouped.get_group(ground_truth)
            sample = group_df.iloc[random.randint(0, len(group_df) - 1)]
            
            # Extract filename from the current path and create new path
            current_path = sample['Filepath']
            filename = os.path.basename(current_path)
            new_path = f"anecdotal/{dataset}/{filename}"
            
            # Add to files to upload
            files_to_upload.append((current_path, new_path))
            
            # Generate the LaTeX output for this sample
            latex_output = f"""
            \\begin{{figure}}[t!]
                \\small
                \\textbf{{{question}}}
                \\vspace{{1em}}
                \\begin{{center}}
                \\includegraphics[height=0.45\\linewidth]{{{new_path}}}
                \\end{{center}}
                \\vspace{{1em}}
                
                \\begin{{tabular}}{{|p{{0.28\\linewidth}}|p{{0.28\\linewidth}}|p{{0.28\\linewidth}}|}}
                    \\hline
                    \\textbf{{Category}} & \\textbf{{Subcategory}} & \\textbf{{Task}} \\\\
                    \\hline
                    {category} & {subcategory} & {dataset} \\\\
                    \\hline
                \\end{{tabular}}
                \\vspace{{1em}}

                \\textbf{{Ground Truth:}} {sample['Ground_Truth']}
                \\vspace{{1em}}

                \\textbf{{Predictions:}}
                \\vspace{{1em}}
                
                \\begin{{tabular}}{{p{{0.28\\linewidth}}|p{{0.28\\linewidth}}|p{{0.28\\linewidth}}}}
                    \\textbf{{Model Name}} & \\textbf{{0 shot}} & \\textbf{{8 shot}} \\\\
                    \\hline
                    Gemini-pro-1.5 & {sample['Gemini-pro-1.5_0-shot']} & {sample['Gemini-pro-1.5_8-shot']} \\\\
                    GPT-4o & {sample['GPT-4o_0-shot']} & {sample['GPT-4o_8-shot']} \\\\
                    LLaVA v1.6 34B & {sample['LLaVA v1.6 34B_0-shot']} & {sample['LLaVA v1.6 34B_8-shot']} \\\\
                    Claude-3.5-sonnet & {sample['Claude-3.5-sonnet_0-shot']} & {sample['Claude-3.5-sonnet_8-shot']} \\\\
                    Gemini-flash-1.5 & {sample['Gemini-flash-1.5_0-shot']} & {sample['Gemini-flash-1.5_8-shot']} \\\\
                    Claude-3-haiku & {sample['Claude-3-haiku_0-shot']} & {sample['Claude-3-haiku_8-shot']}
                \\end{{tabular}}
            \\end{{figure}}
            """
            
            latex_outputs.append(latex_output)
    
    return '\n\n'.join(latex_outputs), files_to_upload

# Assuming question_mapping and dataset_mapping are defined as before

# Generate the LaTeX output and get the list of files to upload
latex_output, files_to_upload = generate_latex_output(sample_predictions, question_mapping, dataset_mapping)

# Print the LaTeX output
print("LaTeX Output:")
print(latex_output)

# Print the list of files to upload
print("\nFiles to Upload:")
for original_path, new_path in files_to_upload:
    print(f"Original: {original_path}")
    print(f"New: {new_path}")
    print()

LaTeX Output:

            \begin{figure}[t!]
                \small
                \textbf{What wheat variety is this?}
                \vspace{1em}
                \begin{center}
                \includegraphics[height=0.45\linewidth]{anecdotal/Durum Wheat/Frame 0026.jpg}
                \end{center}
                \vspace{1em}
                
                \begin{tabular}{|p{0.28\linewidth}|p{0.28\linewidth}|p{0.28\linewidth}|}
                    \hline
                    \textbf{Category} & \textbf{Subcategory} & \textbf{Task} \\
                    \hline
                    Identification (I) & Seed Morphology & Durum Wheat \\
                    \hline
                \end{tabular}
                \vspace{1em}

                \textbf{Ground Truth:} Foreign Matters
                \vspace{1em}

                \textbf{Predictions:}
                \vspace{1em}
                
                \begin{tabular}{p{0.28\linewidth}|p{0.28\linewidth}|p{0.28\linewidth}}
         